In [4]:
#Importing all the necessary packages
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from tqdm import tqdm

In [5]:
#Ignoring all warnings
import warnings
warnings.filterwarnings('ignore')

In [6]:
url = 'https://citizen.mppolice.gov.in/Custom_Arrested_Person.aspx'

In [7]:
list_days = list(range(1,32))
days = ["{:02}".format(num) for num in list_days]

In [8]:
list_months = list(range(1,13))
months = ["{:02}".format(num) for num in list_months]

In [9]:
#Enter range of required years here 
list_years = list(range(2016,2021))
years = ["{:02}".format(num) for num in list_years]

In [10]:
#DON'T CHANGE!
#Function to scrape the first page of every day in a given year
def all_days(url, month, year, district):
    with requests.Session() as req:
        req.headers['user-agent'] = 'Mozilla/5.0'
        r = req.get(url, verify=False)
        soup = BeautifulSoup(r.content, 'lxml')
        df = None
        for day in days:
            date = '{}/{}/{}'.format(day, month, year)
            payload = {
            #     "__LASTFOCUS": ""
                "__EVENTTARGET": "",
                '__EVENTARGUMENT': '',
                "ctl00$hdnSessionIdleTime": "",
                "ctl00$hdnUserUniqueId": "",
                "ctl00$ContentPlaceHolder1$ddlDistrict": "{}".format(district),
                "ctl00$ContentPlaceHolder1$ddlPoliceStation": "",
                "ctl00$ContentPlaceHolder1$txtStartDate": "{}".format(date),
                "ctl00$ContentPlaceHolder1$meeSecurityAnswer1_ClientState": "",
                "ctl00$ContentPlaceHolder1$txtEndDate": "{}".format(date),
                "ctl00$ContentPlaceHolder1$txtEndDate_MaskedEditExtender_ClientState": "",
                "ctl00$ContentPlaceHolder1$txtmissing_name":"",
                "ctl00$ContentPlaceHolder1$btnSearch": "खोजें"

            }
            payload['__VIEWSTATE'] = soup.find("input", id="__VIEWSTATE").get("value")
            payload['__EVENTVALIDATION'] = soup.find(
            "input", id="__EVENTVALIDATION").get("value")
            r = req.post(url, data=payload)
            try:
                df1 = pd.read_html(r.content, attrs={
                              'id': 'ContentPlaceHolder1_GrdFoundPS'})[0]
                df1['date'] = date
                df1['year'] = year  
                if df is None:
                    df = df1
                else:
                    df = df.append(df1)
            except:
                pass 
        return df

In [191]:
#DON'T CHANGE!
def missing_days(url, district, page, missing_dates):
    with requests.Session() as req:
        req.headers['user-agent'] = 'Mozilla/5.0'
        r = req.get(url, verify=False)
        soup = BeautifulSoup(r.content, 'lxml')
        df = None
        for day in tqdm(missing_dates):
            payload = {
            #     "__LASTFOCUS": ""
                "__EVENTTARGET": "ctl00$ContentPlaceHolder1$GrdFoundPS",
                '__EVENTARGUMENT': 'Page${}'.format(page), #Change to 3 when required
                "ctl00$hdnSessionIdleTime": "",
                "ctl00$hdnUserUniqueId": "",
                "ctl00$ContentPlaceHolder1$ddlDistrict": "{}".format(district),
                "ctl00$ContentPlaceHolder1$ddlPoliceStation": "",
                "ctl00$ContentPlaceHolder1$txtStartDate": "{}".format(day),
                "ctl00$ContentPlaceHolder1$meeSecurityAnswer1_ClientState": "",
                "ctl00$ContentPlaceHolder1$txtEndDate": "{}".format(day),
                "ctl00$ContentPlaceHolder1$txtEndDate_MaskedEditExtender_ClientState": "",
                "ctl00$ContentPlaceHolder1$txtmissing_name":"",
#                 "ctl00$ContentPlaceHolder1$btnSearch": "खोजें"

            }
            payload['__VIEWSTATE'] = soup.find("input", id="__VIEWSTATE").get("value")
            payload['__EVENTVALIDATION'] = soup.find(
            "input", id="__EVENTVALIDATION").get("value")
            r = req.post(url, data=payload)
            try:
                df1 = pd.read_html(r.content, attrs={
                              'id': 'ContentPlaceHolder1_GrdFoundPS'})[0]
                df1['date'] = day
                year = re.findall(r'\d+/\d+/(.+)', day)
                df1['year'] = year[0] 
                if df is None:
                    df = df1
                else:
                    df = df.append(df1)
            except:
                pass
#         df['year'] = year    
        return df

In [105]:
def p_ones(district_no):
    district = None
    for year in tqdm(years):
        for month in tqdm(months):
            df = all_days(url, month, year, district_no) #Enter district code here
            if district is None:
                district = df
            else:
                district = district.append(df)
    return district

In [157]:
# def p_rest(district, district_no):
#     zila = district.जिला.values[0]            
#     missing_dates_df = district[district.जिला != zila]
# #     missing_dates = missing_dates_df.date.values
#     p_t = missing_dates_df.जिला.unique()
#     p_n = sorted(p_t, key = len)
#     page = 2
#     md = None
#     for i in range(len(p_n)):
#         missing_dates = missing_dates_df.date.values #new
#         if len(missing_dates) >= 1:
#             df_2 = missing_days(url, district_no, page)
#             page += 1
# #             missing_dates_df = missing_dates_df[missing_dates_df.जिला != p_n[i]]
# #             missing_dates = missing_dates_df.date.values
#             if md is None:
#                 md = df_2
#             else:
#                 md = md.append(df_2)
#         missing_dates_df = missing_dates_df[missing_dates_df.जिला != p_n[i]]
#         missing_dates = missing_dates_df.date.values
#         return md

In [198]:
def p_rest(district, district_no):
    zila = district.जिला.values[0]            
    missing_dates_df = district[district.जिला != zila]
    missing_dates = missing_dates_df.date.values
    p_t = missing_dates_df.जिला.unique()
    p_n = sorted(p_t, key = len)
    sage = 2
    md = None
    for i in range(len(p_n)):
        df_2 = missing_days(url, district_no, sage, missing_dates)
        missing_dates_df = missing_dates_df[missing_dates_df.जिला != p_n[i]]
#         print(len(missing_dates_df))
#         print(p_n[i])
        missing_dates = missing_dates_df.date.values
#         print(len(missing_dates))
        sage += 1
#         print(sage)
        if md is None:
                md = df_2
        else:
                md = md.append(df_2)
    return md

In [212]:
def autoscraper(district_no):
    o = p_ones(district_no)
    m = p_rest(o, district_no)
    om = o.append(m)
    z = om.जिला.values[0]
    om = om[om.जिला == z]
    return om 

In [ ]:
list_of_districts = ['21338','21369','21304',
                    '21315','21357','21370','21350','21351']
for jj in list_of_districts:
    dd = autoscraper(jj)
    dd.to_excel('df_{}.xlsx'.format(jj), index = False)
    dd.to_csv('df_{}.csv'.format(jj), index = False)

 42%|████▏     | 5/12 [07:52<12:31, 107.30s/it]

In [217]:
len(list_of_districts)

12

In [215]:
umaria.drop_duplicates()

,क्र.सं.,जिला,थाना,गिरफ्तार व्यक्ति का नाम,धारा एबं अधिनियम,पूछताछ अधिकारी के नाम,पूछताछ अधिकारी के पद,date,year
0,1,उमरिया,नौरोजाबाद,विक्रम महोबिया,म.प्र. आबकारी अधिनियम 1915-34(1),SHIVSHANKAR SINGH,SI (Sub-Inspector),01/01/2016,2016
0,1,उमरिया,मानपुर,राजेश खटीक,"भा दं सं 1860-294,भा दं सं 1860-323,भा दं सं 1...",SOHAN SINGH,HC (Head Constable),02/01/2016,2016
0,1,उमरिया,चंदिया,बस क्रं.MP54P0203 का चालक पुष्पराज सिंह,"भा दं सं 1860-279,भा दं सं 1860-337",RAJENDRA PRASAD YADAV,Asst. SI (Assistant Sub-Inspector),03/01/2016,2016
1,2,उमरिया,मानपुर,रामसखा यादव,"भा दं सं 1860-451,भा दं सं 1860-294,भा दं सं 1...",SHIV SHANKAR MISHRA,SI (Sub-Inspector),03/01/2016,2016
0,1,उमरिया,चंदिया,मनोज यादव,भा दं सं 1860-304-A,RAJENDRA PRASAD YADAV,Asst. SI (Assistant Sub-Inspector),04/01/2016,2016
...,...,...,...,...,...,...,...,...,...
10,71,उमरिया,उमरिया,अनिल कोल उम्र 19 वर्ष,"पब्लिक गेब्लिंग एक्ट, 1867-13",shashank rana,SI (Sub-Inspector),08/11/2018,2018
11,72,उमरिया,उमरिया,रज्जन कोल उम्र 27 वर्ष,"पब्लिक गेब्लिंग एक्ट, 1867-13",shashank rana,SI (Sub-Inspector),08/11/2018,2018
12,73,उमरिया,उमरिया,आशीष हरिजन उम्र 26 वर्ष,"पब्लिक गेब्लिंग एक्ट, 1867-13",shashank rana,SI (Sub-Inspector),08/11/2018,2018
13,74,उमरिया,उमरिया,विकास कुमार उम्र 19 वर्ष,"पब्लिक गेब्लिंग एक्ट, 1867-13",shashank rana,SI (Sub-Inspector),08/11/2018,2018


In [91]:
katni = katni.rename(columns={'जिला':'District','थाना':'PS','गिरफ्तार व्यक्ति का नाम':'Name',
                              'धारा एबं अधिनियम':'Offence','पूछताछ अधिकारी के नाम':'IO',
                             'पूछताछ अधिकारी के पद':'Designation'})

In [92]:
katni['last_name'] = katni['Name'].str.split().str[-1]

In [93]:
katni.drop('क्र.सं.', axis=1, inplace=True)

In [94]:
#Saving to csv and excel
katni.to_excel('katni.xlsx', index = False)
katni.to_csv('katni.csv', index = False)

In [95]:
katni.to_excel(r'C:\Users\sami\Documents\Python Scripts\CPA Project\WPA\katni.xlsx', index = False)

'अशोक नगर'